In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import matplotlib.pyplot as plt


from ipynb.fs.full.fonctions import boundingBoxes,display,drawContour,checkBoxes,substraction
from ipynb.fs.full.wavelet import middle,Wavelet
from ipynb.fs.full.ransac import doRansac,plotLines
from ipynb.fs.full.white_select import selectWhite,selectWhiteHSV

c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\lc100\AppData\Local\miniconda3\envs\frameSubtraction\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def detect_ball(model, frame: np.ndarray):
    WIDTH = 360
    HEIGHT = 360
    resized_frame = cv2.resize(frame, (WIDTH, HEIGHT))
    detection = model(resized_frame)
    bounding_box = detection.xyxy[0].numpy()

    for box in bounding_box:
        if box[5] == 0:
            x = box[0] + (box[2] - box[0]) / 2
            y = box[1] + (box[3] - box[1]) / 2
            return (int(x), int(y)), (box[0], box[1], box[2], box[3])
            # array([     124.15,         179,      130.78,      185.98,      0.8651,           0], dtype=float32)
            # box[0]:   Left
            # box[1]:   Top
            # box[2]:   Right
            # box[3]:   Bottom
            # box[4]:   Probability
            # box[5]:   Klasse 0 = Ball

    return False, False

ball_model = torch.hub.load('ultralytics/yolov5', 'custom', path='03_Ball_Detection/models/ball_weights_V2.pt', force_reload=False)

def detect_ball_static(image,model=ball_model):

    height, width, _ = image.shape

    # start object detection
    # ball detection returns center coordinates from the ball
    ball_center, ball_bb = detect_ball(model, image)
    if ball_center:  
        ball_center = [ball_center[0] * (width / 360), ball_center[1] * (height / 360)] 
        ball_bb=[int(ball_bb[0]* (width / 360)),
                int(ball_bb[1]* (height / 360)),
                int(ball_bb[2]* (width / 360)),
                int(ball_bb[3]* (height / 360))]  
        
        return [ball_bb[0],ball_bb[1],ball_bb[2]-ball_bb[0]+10,ball_bb[3]-ball_bb[1]+10]
    return None

Using cache found in C:\Users\lc100/.cache\torch\hub\ultralytics_yolov5_master
requirements: Ultralytics requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
requirements:  AutoUpdate skipped (offline)
YOLOv5  2023-6-14 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 


In [3]:

%matplotlib qt

In [4]:
# 2 frames for the substraction
frame1=None #newest frame
frame0=None #previous frame

# 1st frame will be use to detect the white borders of the field
first_frame=None
white_border=None

# index is the time of the frame
idx=0
# time jump between 2 frame
jump=100

#3D points (x,y,t,type)
position=[[0,0,0,0]]
#number of pts used for RANSAC
size_slice=10
#pts after ransac
corrected=np.array([[0,0,0,0]])
#pts after correction of the trajectory
corrected2=np.array([[0,0,0,0]])
# index of the last pts that had is trajectory corrected in corrected[]
lastcorrected=0 
firstLoop=True
lastLen=0

# all the lines found by RANSAC
all_lines=[ [ [[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]] ] ]
#all_lines=[[[0,0,0],[0,0,0]]]

model=None
lastPosition=None

static=False
static_box=None

#templates used for the wavelet function
template = cv2.imread('template/templateRGB_Black.png')
template_HSV_W= cv2.cvtColor(template,cv2.COLOR_BGR2HSV)

templateB = cv2.imread('template/templateB.png')
template_HSV_B = Wavelet(templateB)

#video
cap = cv2.VideoCapture('video_record/1310.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")


In [5]:
#Read until video is completed
while(cap.isOpened()):
    
    # Capture frame-by-frame
    ret, frame_og= cap.read()
    
    if ret == True:
        model=None
        
        frame_og_crop=frame_og[:,:1700,:].copy()
        last_frame = frame_og_crop.copy()

        #detect the white border so to not confuse them with the ball
        if first_frame is None:
            frame0=frame_og_crop.copy()
            frame1=frame_og_crop.copy()
            
            first_frame = frame_og_crop.copy()
            white_border=selectWhite(first_frame,dilate=9,erode=3)

            white_border=cv2.bitwise_not(white_border)

            display(white_border,name='not border')

    
        frame0 = frame1.copy()
        frame1 = frame_og_crop.copy()

        #white select
        #maskW1=selectWhite(frame1,erode=3,dilate=8,B_limit=70)
        #maskW0=selectWhite(frame0,erode=3,dilate=8,B_limit=70)

        #maskW=cv2.bitwise_or(maskW1, maskW0, mask = None)
        #maskW=cv2.bitwise_and(maskW,white_border, mask=None)
        #maskW = cv2.erode(maskW, None, iterations=4)
        #maskWA = cv2.dilate(maskW, None, iterations=15)
        
        #frame_white=cv2.bitwise_and(frame_og_crop.copy(),frame_og_crop.copy(), mask=maskW)
        #display(frame_white,name='white mask RGB')
        
        
        #substraction
        frameHLS0=cv2.cvtColor(frame0,cv2.COLOR_BGR2YUV)
        frameHLS1=cv2.cvtColor(frame1,cv2.COLOR_BGR2YUV)
        frame_substraction=substraction(frameHLS0[:,:,0],frameHLS1[:,:,0],blur_type=0,threshold_type=0,threshold=30,blur=21,show=True,erode=4,dilate=5)
        
        #white select
        maskedW_sub=cv2.bitwise_and(frame1, frame1, mask=frame_substraction)
        maskedW_sub=selectWhite(maskedW_sub,erode=3,dilate=8,B_limit=0,W_limit=100,H_limit=40)

        maskW = cv2.bitwise_and(maskedW_sub , white_border, mask=None)
        maskW = cv2.erode(maskW, None, iterations=4)
        maskW = cv2.dilate(maskW, None, iterations=10)
        
        frame_white=cv2.bitwise_and(frame_og_crop.copy(),frame_og_crop.copy(), mask=maskW)
        display(frame_white,name='white mask sub RGB')
        
        #whiteANDsub=cv2.bitwise_and(frame_substraction,maskW, mask=None)
        #display(whiteANDsub,name='both sub and white')
        
        #draw contour on frame
        dil,contours=drawContour(maskW,kernelsize=15)
        display(dil,name='dilatation')
        #hide non-contoured area on RGB
        masked1=cv2.bitwise_and(frame1, frame1, mask=dil)
        #put boxes around objectsq
        _,boundRect=boundingBoxes(dil,contours,show=True)
        #detect the ball in one of the boxes (if any)
        _, box = checkBoxes(frame_og_crop.copy(),boundRect,coeff=10,wave_temp=template_HSV_W,wave_temp2=template_HSV_B)
        
        #static detection
        if (box is None and not static) : #did not detected moving ball, static mode on
            static=True    
            box=detect_ball_static(frame_og_crop.copy(),ball_model)     
            static_box= box
        elif box is None and static_box is not None: # still not moving, use the last detection
            box=static_box
        else: # moving, static mode off
            static_box=None
            static=False   
        
        # add the 3D to the points
        if box is not None:
            pts=middle(box)
            
            # if same position, don't take it.
            if not np.array_equal(pts, position[-1][:2]) and ((pts[0]-position[-1][0])**2+(pts[1]-position[-1][1])**2)>100:
                if static:
                    pts=[pts[0],pts[1],idx,1]
                else:
                    pts=[pts[0],pts[1],idx,0]
                position=np.append(position,[pts],axis=0) 
                idx+=jump

            cv2.rectangle(frame_og_crop, (int(box[0]), int(box[1])),(int(box[0]+box[2]), int(box[1]+box[3])), (255,255,255), 2) 


        #Ransac
        Len=len(position)
        if Len%size_slice==0 and lastLen!=Len:
            if firstLoop:
                position=position[1:]

            sliced=position[(Len-size_slice):]

            #if model is not None:
            #    sliced=np.concatenate((corrected[len(corrected)-2:],sliced))

            model,droite=doRansac(sliced,frame_og_crop,distMin=15,D3=True)
            lastLen=Len

            if model is not None:
                corrected=np.concatenate((corrected,model))
                all_lines=np.append(all_lines,[droite],axis=0)

                if firstLoop:
                    position=position[1:]
                    corrected=corrected[1:]
        
        # correct trajectory: if i is shorter to go to the 5th pts rather than the next then we jump
        k=lastcorrected
        lenCorrect=len(corrected)
        
        while k < (lenCorrect-5) :
            diff1=cv2.absdiff(corrected[k],corrected[k+1])
            diff2=cv2.absdiff(corrected[k],corrected[k+5])
            corrected2=np.append(corrected2,[corrected[k]],axis=0)

            if diff1[0]+diff1[1]*1 < (diff2[0]+diff2[1]):
                k+=1
            else:
                k+=4
        if firstLoop and model is not None:
            corrected2=corrected2[1:]
            firstLoop=False
        if lenCorrect>5:
            lastcorrected = lenCorrect-5


        #plot
        for index, item in enumerate(position): 
            if not item[3]: #green is moving
                cv2.circle(frame_og_crop, item[:2], 2, [20, 255, 20], 5)
            else: # blue is static
                cv2.circle(frame_og_crop, item[:2], 2, [255, 200, 50], 5)

        plotLines(corrected,lines2=corrected2,frame=frame_og_crop)

        display(frame_og_crop,name='detection')
    
        #press key to go the next frame or Q to exit or S to save the frame
        key = cv2.waitKey(0)
        
        if key == ord('q') or key == ord('Q'):
            break
        elif key == ord('s') or key == ord('S'):
            box_cp = frame_og[box[1]:(box[1]+box[3]),box[0]:(box[0]+box[2]),:]
            cv2.imwrite("object1_{}.png".format(idx), box_cp)
        else:
            continue
            
  # Break the loop
    else: 
        break
    

all_lines=all_lines[1:]



2.6666666666666665
3.5833333333333335
2.1666666666666665
3.25
3.1666666666666665
3.9166666666666665
3.25
3.4166666666666665
3.75
3.3333333333333335
4.083333333333333
3.9166666666666665
2.6666666666666665
3.8333333333333335
3.5833333333333335
4.166666666666667
4.75
4.75
4.916666666666667
4.333333333333333
4.5
3.5
3.5833333333333335
2.75
1.8333333333333333
2.75
4.25
3.8333333333333335
3.5833333333333335
4.083333333333333
4.333333333333333
2.8333333333333335
2.4166666666666665
3.1666666666666665
3.0
2.0833333333333335
2.5
2.5833333333333335
3.25
2.75
2.8333333333333335
2.5
4.416666666666667
2.25
2.8333333333333335
4.166666666666667
2.75
2.5
2.5833333333333335
2.75
3.1666666666666665
2.9166666666666665
2.5833333333333335
3.0
3.9166666666666665
4.916666666666667
4.583333333333333
3.5833333333333335
4.75
4.666666666666667
4.666666666666667
3.5833333333333335
3.0
2.75
3.0833333333333335
3.0833333333333335
3.75
3.25
2.0833333333333335
2.4166666666666665
2.6666666666666665
3.0833333333333335
2.

In [6]:

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111, projection='3d')
ax.grid()

ax.plot3D(corrected[:,1],corrected[:,0],corrected[:,2],'r')
ax.scatter(position[:,1],position[:,0],position[:,2],marker="+")

for item in all_lines:
     start_pts=item[0][0]
     middle_pts=item[0][1]
     end_pts=item[1][1]
     
     ax.scatter(middle_pts[1],middle_pts[0],middle_pts[2],color='k')
     ax.scatter(end_pts[1],end_pts[0],end_pts[2],color='g')
     ax.scatter(start_pts[1],start_pts[0],start_pts[2],color='g')
     
     ax.plot([start_pts[1], middle_pts[1]], [start_pts[0], middle_pts[0]], zs=[start_pts[2], middle_pts[2]],color='k')
     ax.plot([end_pts[1], middle_pts[1]], [end_pts[0], middle_pts[0]], zs=[end_pts[2], middle_pts[2]],color='k')


ax.set_ylabel('x', labelpad=20)
ax.set_ylim(0, 1500)
ax.set_xlabel('y', labelpad=20)
ax.set_xlim(0, 1500)
ax.set_zlabel('t', labelpad=20)
#ax.set_zlim(0, 100)

fig = plt.figure(figsize = (8,8))

plt.plot(corrected[:,2],corrected[:,1],'g')
plt.plot(corrected[:,2],corrected[:,0],'r')
plt.plot(corrected[:,2],corrected[:,1],'g+')
plt.plot(corrected[:,2],corrected[:,0],'r+')

fig = plt.figure(figsize = (8,8))

plt.plot(corrected2[:,2],corrected2[:,1],'k')
plt.plot(corrected2[:,2],corrected2[:,0],'b')

plt.show()

plotLines(corrected,corrected2,frame_og_crop)

key = cv2.waitKey(0)
#When everything done, release the video capture object
cap.release()  
# Closes all the frames
cv2.destroyAllWindows()

In [7]:
%matplotlib inline

In [8]:
"""
#classifier = keras_cv.models.ImageClassifier.from_preset(
#    "efficientnetv2_b0_imagenet_classifier"
#)

image = keras.utils.load_img("Imageterrain.jpg")
image = np.array(image)
keras_cv.visualization.plot_image_gallery(
    [image], rows=1, cols=1, value_range=(0, 255), show=True, scale=4
)
predictions = classifier.predict(np.expand_dims(image, axis=0))
top_classes = predictions[0].argsort(axis=-1)
classes = keras.utils.get_file(
    origin="https://gist.githubusercontent.com/LukeWood/62eebcd5c5c4a4d0e0b7845780f76d55/raw/fde63e5e4c09e2fa0a3436680f436bdcb8325aac/ImagenetClassnames.json"
)
with open(classes, "rb") as f:
    classes = json.load(f)
top_two = [classes[str(i)] for i in top_classes[-2:]]
print("Top two classes are:", top_two)
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
tfds.disable_progress_bar()

data, dataset_info = tfds.load("cats_vs_dogs", with_info=True, as_supervised=True)
train_steps_per_epoch = dataset_info.splits["train"].num_examples // BATCH_SIZE
train_dataset = data["train"]

num_classes = dataset_info.features["label"].num_classes

resizing = keras_cv.layers.Resizing(
    IMAGE_SIZE[0], IMAGE_SIZE[1], crop_to_aspect_ratio=True
)


def preprocess_inputs(image, label):
    image = tf.cast(image, tf.float32)
    # Staticly resize images as we only iterate the dataset once.
    return resizing(image), tf.one_hot(label, num_classes)


# Shuffle the dataset to increase diversity of batches.
# 10*BATCH_SIZE follows the assumption that bigger machines can handle bigger
# shuffle buffers.
train_dataset = train_dataset.shuffle(
    10 * BATCH_SIZE, reshuffle_each_iteration=True
).map(preprocess_inputs, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)

images = next(iter(train_dataset.take(1)))[0]
keras_cv.visualization.plot_image_gallery(images, value_range=(0, 255))
#setting the path to the directory containing the pics
path = 'G:\\06  Projekte\\72 KI\\01 Studenten\\2023_06_Couture\\ball_tracking\\object_detected'
#appending the pics to the training data list
training_data = [] 
borderType = cv2.BORDER_CONSTANT
value = [0, 0, 0]
size=64

for img in os.listdir(path):
    pic = cv2.imread(os.path.join(path,img))
    """"""
    more=(size- pic.shape[0])
    top = 0
    bottom = more
    more1=(size- pic.shape[1])
    left = 0
    right = more1
    
    a = cv2.copyMakeBorder(pic, top, bottom, left, right, borderType,value=value)
    """"""
    
    pic = cv2.resize(pic,(size,size))
    training_data.append([pic])
    
#converting the list to numpy array and saving it to a file using #numpy.save
np.save(os.path.join("./",'aaa_false_set'),np.array(training_data))
#loading the saved file once again
saved = np.load(os.path.join("./",'aaa_data_set.npy'))
print(saved.shape)
plt.imshow(saved[0].reshape(size,size,3))
plt.imshow(np.array(training_data[0]).reshape(size,size,3))
"""


'\n#classifier = keras_cv.models.ImageClassifier.from_preset(\n#    "efficientnetv2_b0_imagenet_classifier"\n#)\n\nimage = keras.utils.load_img("Imageterrain.jpg")\nimage = np.array(image)\nkeras_cv.visualization.plot_image_gallery(\n    [image], rows=1, cols=1, value_range=(0, 255), show=True, scale=4\n)\npredictions = classifier.predict(np.expand_dims(image, axis=0))\ntop_classes = predictions[0].argsort(axis=-1)\nclasses = keras.utils.get_file(\n    origin="https://gist.githubusercontent.com/LukeWood/62eebcd5c5c4a4d0e0b7845780f76d55/raw/fde63e5e4c09e2fa0a3436680f436bdcb8325aac/ImagenetClassnames.json"\n)\nwith open(classes, "rb") as f:\n    classes = json.load(f)\ntop_two = [classes[str(i)] for i in top_classes[-2:]]\nprint("Top two classes are:", top_two)\nBATCH_SIZE = 32\nIMAGE_SIZE = (224, 224)\nAUTOTUNE = tf.data.AUTOTUNE\ntfds.disable_progress_bar()\n\ndata, dataset_info = tfds.load("cats_vs_dogs", with_info=True, as_supervised=True)\ntrain_steps_per_epoch = dataset_info.splits